In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb._debug.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'bert-base-uncased',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [7]:
from typing import Generator
import torch
import numpy as np
import pandas as pd
from scipy.spatial import distance


def analysis(
        data: pd.DataFrame,
        dim: list = None
) -> Generator:
    if dim is None:
        dim = [768, 384, 192, 96, 48, 24, 12, 6, 3]

    embed_col: np.ndarray = torch.stack(data[enc.col_name].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == enc.dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        # compute centroid means and dispersion for each cluster
        for label, group in data.groupby(dataset.target_label):
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

        record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

        yield record

In [8]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis

,dim,centroid_negative,dispersion_negative,centroid_positive,dispersion_positive,distance
0,768,"[-0.05271393805742264, -0.11926598846912384, 0...",2937.050055,"[-0.15186531841754913, -0.03394192084670067, 0...",740.529710,3.035837
1,384,"[-0.0768522068157001, 0.06856719979299332, -0....",2939.398846,"[0.1596161218479925, -0.14240879957006303, 0.1...",741.410423,3.001051
2,192,"[-0.07073982320859394, -0.08127288837630418, 0...",2938.948498,"[0.14692117127938747, 0.16879753739693953, -0....",741.359616,3.009252
3,96,"[0.05715493972521865, 0.13546924826397033, -0....",2938.800238,"[-0.11870641327545414, -0.28135920793286145, 0...",741.244524,3.008571
4,48,"[-0.09715416259024226, 0.0020771797676757985, ...",2936.558992,"[0.2017817223028109, -0.00431414259440368, 0.4...",739.830286,3.059979
5,24,"[-0.07795996254416153, -0.10610419304787336, -...",2934.177324,"[0.16191684528402764, 0.22037024709942893, 0.2...",740.455521,3.008284
6,12,"[-0.03868075811875576, -0.24673964505329657, -...",2924.989698,"[0.08033695916972322, 0.5124592628030007, 0.87...",739.991042,3.026398
7,6,"[0.2605781725061415, -0.09129301313947706, 0.0...",2892.116361,"[-0.541200819820448, 0.1896085657512215, -0.18...",741.108529,2.802988
8,3,"[-0.5601945865773238, 0.3940483758749429, -0.7...",2784.198116,"[1.1634810644298275, -0.8184081652787278, 1.51...",727.061534,3.080517


In [9]:
cluster_analysis.to_csv[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]('./cluster_analysis.csv')